In [2]:
import urllib3
from bs4 import BeautifulSoup
import requests
import certifi

In [3]:
#!/usr/bin/python
import MySQLdb

db = MySQLdb.connect(host="localhost",    # your host, usually localhost
                     user="root",         # your username
                     #passwd="xpx",  # your password
                     db="xudatabase")        # name of the data base

cur = db.cursor()

In [12]:
dailyMax = 25
batchNum =10

startingPMID=27694200 #get from database and passed in as parameter
endingPMID = startingPMID

In [19]:
for i in range (startingPMID, startingPMID+dailyMax, batchNum):
    PMIDs = ""
    for j in range (batchNum):
        PMIDs = PMIDs + str(startingPMID + i + j ) +","
        
    url = "https://www.ncbi.nlm.nih.gov/pubmed/"+ PMIDs +"?report=medline&format=text" 
    print url
    
    

https://www.ncbi.nlm.nih.gov/pubmed/55388400,55388401,55388402,55388403,55388404,55388405,55388406,55388407,55388408,55388409,?report=medline&format=text
https://www.ncbi.nlm.nih.gov/pubmed/55388410,55388411,55388412,55388413,55388414,55388415,55388416,55388417,55388418,55388419,?report=medline&format=text
https://www.ncbi.nlm.nih.gov/pubmed/55388420,55388421,55388422,55388423,55388424,55388425,55388426,55388427,55388428,55388429,?report=medline&format=text


In [15]:
%%time
for i in range (startingPMID, startingPMID+dailyMax, batchNum):
    PMIDs = ""
    for j in range (batchNum):
        PMIDs += str(startingPMID + j ) 
        
    url = "https://www.ncbi.nlm.nih.gov/pubmed/"+ PMIDs +"?report=medline&format=text" 
    
    
    http = urllib3.PoolManager(cert_reqs='CERT_REQUIRED',ca_certs=certifi.where())
    r = http.request('GET', url)
    soup = BeautifulSoup(r.data, "lxml")
       
    allInfo = unicode(soup.pre.string)
    errorNum =0
    
    if allInfo.find('Error occurred: The following PMID is not available')>0:
        print 'error'
        errorNum +=1
        if errorNum>10:
            break
            
    if allInfo.find('AB  -')<0:
        print 'no Abstract'
        continue

    IdIndexStart = allInfo.index('PMID')+6
    IdIndexEnd = allInfo.index('\n', IdIndexStart)
    PMID = allInfo[IdIndexStart:IdIndexEnd]
    
    abIndexStart = allInfo.index('AB  -')+6
    abIndexEnd = allInfo.index('\n', abIndexStart)
    ab = allInfo[abIndexStart:abIndexEnd]
    
    query = "insert into articles values ("+ PMID +", '"+ ab + "')"

    try:
        cur.execute(query)
        db.commit()

    except:
        db.rollback()
    
    
    endingPMID = PMID    
    print endingPMID
    

27694200
27694201
CPU times: user 31.3 ms, sys: 3.55 ms, total: 34.9 ms
Wall time: 1.73 s


In [15]:
# print all the first cell of all the rows
# for row in cur.fetchall():
#     print row
db.close()